In [2]:
from pathlib import Path
from dotenv import dotenv_values
def read_envfile(path:Path):
    env_vars = dict(dotenv_values(path))
    return env_vars

In [6]:
read_envfile(Path("../embedding.env"))

{'"SAGEMAKER_MODEL_SERVER_WORKERS"': '1',
 '"TS_DEFAULT_WORKERS_PER_MODEL"': '1'}

In [2]:
properties = {"engine": "MPI", "option.model_id": "tiiuae/falcon-7b-instruct", "option.trust_remote_code": "true", "option.tensor_parallel_degree": 1, "option.paged_attention": "true", "option.max_rolling_batch_size": 64, "option.rolling_batch": "lmi-dist", "option.max_rolling_batch_prefill_tokens": 1560}

In [6]:
with open("../falcon-7b-instruct/serving.properties", "w") as f:
    for pro in properties.items():
        p,v = pro
        f.write(f"{p}={v} \n")
    

In [2]:
import boto3
import json
smr_client =  boto3.client("sagemaker-runtime")
endpoint_name = "lmi-model-2025-05-15-01-57-52-403"

In [3]:
smr_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=json.dumps(
        {
            "inputs": "The diamondback terrapin was the first reptile to be",
            "parameters": {
                "do_sample": True,
                "max_new_tokens": 256,
                "min_new_tokens": 256,
                "temperature": 0.3,
                "watermark": True,
            },
        }
    ),
    ContentType="application/json",
)["Body"].read().decode("utf8")

'{"generated_text": " listed as endangered in the United States. The diamondback terrapin is a small turtle found only in the coastal bays of the Atlantic coast of the United States. The diamondback terrapin is a land turtle that lives in brackish water. It is the only turtle in the world that can swim in both fresh and salt water.\\nThe diamondback terrapin is a small turtle that is found only in the coastal bays of the Atlantic coast of the United States. The diamondback terrapin is a land turtle that lives in brackish water. It is the only turtle in the world that can swim in both fresh and salt water.\\nThe diamondback terrapin is a small turtle that is found only in the coastal bays of the Atlantic coast of the United States. The diamondback terrapin is a land turtle that lives in brackish water. It is the only turtle in the world that can swim in both fresh and salt water. The diamondback terrapin is a small turtle that is found only in the coastal bays of"}'

In [5]:
import os
os.getcwd()

'/home/mrafi/Desktop/Books/RAGwithSagemaker/research'

In [8]:
os.chdir("../")

In [9]:
os.getcwd()

'/home/mrafi/Desktop/Books/RAGwithSagemaker'

In [10]:
import os
import pickle
import re
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from langchain.vectorstores import Chroma, AtlasDB, FAISS
from RAGwithSagemaker.logging.logging import logger

def vectorizedocs(embeddings):
    logger.info("starting docs  loadig")
    loader = PyPDFDirectoryLoader("RAGwithSagemaker/data")
    docs = loader.load()
    logger.info("docs loaded")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
    final_docs = text_splitter.split_documents(docs)
    logger.info("starting vector dbs")
    vector_store = FAISS.from_documents(final_docs, embeddings)
    # with open("vector_store.pth", "wb") as file:           # Optional. Helps in resuing the vectorspace directly without processing the files everytime
    #     vector_store = torch.save(vector_store,file)
    return vector_store

In [11]:
from RAGwithSagemaker.config.configuration import ConfigurationManager
from RAGwithSagemaker.cloud.embeddingmodel import DeployEmbeddingModel
from RAGwithSagemaker.cloud.textgenerationmodel import DeployTextGenerationModel
from RAGwithSagemaker.cloud.ragendpoints import RAGEndPoints
from RAGwithSagemaker.cloud.vectorize_docs import vectorizedocs
from RAGwithSagemaker.logging.logging import logger


congfiguration = ConfigurationManager()
sagemaker_config = congfiguration.get_sagemakersession_config()
embeddings_config = congfiguration.get_embeddings_config()
textgeneration_config = congfiguration.get_textgeneration_config()
s3_config = congfiguration.get_s3_config()
rag_config = congfiguration.get_rag_config()

# text_model_deploy =DeployTextGenerationModel(sagemaker_config, textgeneration_config)
# text_model_deploy.creat_and_deploy_model()

# embedding_model_deploy = DeployEmbeddingModel(sagemaker_config,embeddings_config )
# embedding_model_deploy.deploy_embedding_model()

rag_endpoints = RAGEndPoints(rag_config)
embeddings_endpoint, sm_llm_endpoint = rag_endpoints.create_rag_endpoints()

logger.info("Started vector db")
vector_store = vectorizedocs(embeddings_endpoint)

2025-05-14 21:54:07,149, common.py, 19, INFO, config/config.yaml file loaded sucessfully
2025-05-14 21:54:07,152, common.py, 19, INFO, params.yaml file loaded sucessfully
2025-05-14 21:54:07,153, common.py, 19, INFO, schema.yaml file loaded sucessfully
2025-05-14 21:54:07,155, common.py, 32, INFO, Directory model created
2025-05-14 21:54:07,233, 3111586173.py, 25, INFO, Started vector db
2025-05-14 21:54:07,234, vectorize_docs.py, 11, INFO, starting docs  loadig


{'SAGEMAKER_MODEL_SERVER_WORKERS': '1', 'TS_DEFAULT_WORKERS_PER_MODEL': '1'}


2025-05-14 21:54:13,962, vectorize_docs.py, 14, INFO, docs loaded
2025-05-14 21:54:13,988, vectorize_docs.py, 17, INFO, starting vector dbs


In [15]:
import os
import pickle
import re
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import streamlit as st
import torch


ImportError: cannot import name 'create_stuff_documents_chain' from 'langchain.chains.combine_documents' (/home/mrafi/miniconda3/envs/rag/lib/python3.10/site-packages/langchain/chains/combine_documents/__init__.py)